# HW#1 Regularization

안녕하세요, 광운대학교 로봇학부의 오정현 교수입니다. 본 자료는 딥러닝 실습 수업을 위해 제작된 것입니다.

파이썬 문법
- 점프투파이썬(https://wikidocs.net/book/1) 참고

이번 과제는 딥러닝의 일반화 성능을 높이기 위한 Regularization을 해보는 것입니다.이미지 분류에 여러 가지 Regularization 기법을 적용해 보도록 하겠습니다. 대표적인 Regularization 기법으로 Dropout, Data augmentation, Batch Normalization 등이 있습니다.

이번 과제는 (https://www.tensorflow.org/tutorials/keras/classification?hl=ko)를 참고하면 좋습니다.

#1. Data Generation
Data는 mnist dataset을 이용하도록 하겠습니다. mnist dataset은 원래 60000개의 training set이 주어져 있지만 overfitting을 유도하기 위하여 1000개의 data만 이용하려고 합니다. 1000개의 data로 이루어진 x_train과 y_train을 만들어 보세요. 그리고 2000개로 이루어진 large_x_train, large_y_train을 만들어보세요. 그리고 training data의 다른 범위에서 200개로 이루어진 x_validation과 y_validation도 만들어 보세요.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Flatten
from keras import backend as K
from matplotlib import pyplot

batch_size = 28
num_classes = 10
epochs = 100

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(X_train, Y_train), (x_test,y_test) = mnist.load_data()

### START CODE HERE ###
x_validation = X_train[:200]
y_validation = Y_train[:200]
large_x_train = X_train[200:2200]
large_y_train = Y_train[200:2200]
x_train = X_train[2200:3200]
y_train = Y_train[2200:3200]
### END CODE HERE ###
print("x_validation shape:", x_validation.shape)
print("y_validation shape:", y_validation.shape)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], 1, img_rows, img_cols)
    x_validation = x_validation.reshape(x_validation.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], img_rows, img_cols, 1)
    x_validation = x_validation.reshape(x_validation.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
large_x_train = large_x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
large_x_train /= 255
x_validation /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
large_y_train = keras.utils.to_categorical(large_y_train, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

assert large_x_train.shape[0]==2000
assert large_y_train.shape[0]==2000
assert x_train.shape[0]==1000
assert y_train.shape[0]==1000
assert x_validation.shape[0]==200
assert y_validation.shape[0]==200

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("large_x_train shape:", large_x_train.shape)
print("large_y_train shape:", large_y_train.shape)
print("x_validation shape:", x_validation.shape)
print("y_validation shape:", y_validation.shape)

11490434/11490434 [==============================] - 2s 0us/step
x_validation shape: (200, 28, 28)
y_validation shape: (200,)
x_train shape: (1000, 28, 28, 1)
y_train shape: (1000, 10)
large_x_train shape: (2000, 28, 28, 1)
large_y_train shape: (2000, 10)
x_validation shape: (200, 28, 28, 1)
y_validation shape: (200, 10)


#2. 모델 생성
복습 차원에서 MLP 분류모델을 만들어 보도록 하겠습니다. 모델의 마지막 레이어에는 활성화 함수로 10개의 출력을 가지는 softmax를 달겠습니다. 이를 통해서 모델은 이미지안의 숫자가 0부터 9까지의 숫자중에 어디에 가까운지를 확률적으로 나타냅니다. 



다음과 같은 MLP 모델을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| Dense | (None, 1024) | 1049600 |
| Dense | (None, 1024) | 1049600 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [2]:
model = Sequential()
### START CODE HERE ###
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
### END CODE HERE ###

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 1024)              803840    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 10)                10250     
                                                                 
Total params: 2,913,290
Trainable params: 2,913,290
Non-

#3. Learning MLP
기본 MLP 분류모델을 학습해 보겠습니다. Overfitting은 Training data에 맞추어 과도하게 학습이 이루어져 Test data에서 높은 성능이 나지 않는 현상, 즉 Generalization 성능이 높지 않게 나타나는 현상을 의미합니다. 따라서 Overfitting이 발생하면 Training accuracy는 높지만 Test accuracy는 높지 않게 나타납니다. 

현재 모델이 overfitting이 발생하는지 체크해 보세요. 일부러 overfitting이 발생하도록 유도하였기 때문에 overfitting 현상이 나타나야 합니다.

In [3]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
weights = model.get_weights()

history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
36/36 [==============================] - 5s 30ms/step - loss: 2.4510 - accuracy: 0.0560 - val_loss: 2.3875 - val_accuracy: 0.0900
Epoch 2/100
36/36 [==============================] - 1s 16ms/step - loss: 2.3595 - accuracy: 0.0850 - val_loss: 2.3016 - val_accuracy: 0.1200
Epoch 3/100
36/36 [==============================] - 0s 11ms/step - loss: 2.2753 - accuracy: 0.1330 - val_loss: 2.2227 - val_accuracy: 0.1550
Epoch 4/100
36/36 [==============================] - 0s 10ms/step - loss: 2.1964 - accuracy: 0.1960 - val_loss: 2.1489 - val_accuracy: 0.2100
Epoch 5/100
36/36 [==============================] - 0s 7ms/step - loss: 2.1219 - accuracy: 0.2680 - val_loss: 2.0793 - val_accuracy: 0.3050
Epoch 6/100
36/36 [==============================] - 0s 8ms/step - loss: 2.0512 - accuracy: 0.3410 - val_loss: 2.0139 - val_accuracy: 0.3850
Epoch 7/100
36/36 [==============================] - 0s 7ms/step - loss: 1.9838 - accuracy: 0.4200 - val_loss: 1.9514 - val_accuracy: 0.4350
Epoch 8/1

#4. Regularization
Overfitting이 발생한 모델에 다양한 Regularization 기법을 이용해 보도록 합시다.

4.1 Large Dataset

Training data가 충분하다면 overfitting 현상이 발생할 가능성이 줄어듭니다. 기본 MLP 분류 모델에서 large_x_train과 large_y_train을 이용하면 성능이 올라갈 것입니다. Generalization 성능이 올라갔나요?

In [4]:
#initialize weights
model.set_weights(weights)

large_model_history=model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = model.evaluate(x_test, y_test, verbose=0)
print('Largemodel Test loss:', score[0])
print('Largemodel Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 0s 4ms/step - loss: 2.3529 - accuracy: 0.1050 - val_loss: 2.2211 - val_accuracy: 0.1700
Epoch 2/100
72/72 [==============================] - 0s 4ms/step - loss: 2.1304 - accuracy: 0.2710 - val_loss: 2.0320 - val_accuracy: 0.3600
Epoch 3/100
72/72 [==============================] - 0s 4ms/step - loss: 1.9566 - accuracy: 0.4445 - val_loss: 1.8789 - val_accuracy: 0.5250
Epoch 4/100
72/72 [==============================] - 0s 4ms/step - loss: 1.8123 - accuracy: 0.5655 - val_loss: 1.7506 - val_accuracy: 0.6400
Epoch 5/100
72/72 [==============================] - 0s 4ms/step - loss: 1.6890 - accuracy: 0.6275 - val_loss: 1.6407 - val_accuracy: 0.6800
Epoch 6/100
72/72 [==============================] - 0s 4ms/step - loss: 1.5826 - accuracy: 0.6695 - val_loss: 1.5454 - val_accuracy: 0.7050
Epoch 7/100
72/72 [==============================] - 0s 4ms/step - loss: 1.4891 - accuracy: 0.7025 - val_loss: 1.4618 - val_accuracy: 0.7150
Epoch 8/100
7

##4.2 Dropout
Dropout은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 Dropout layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 Dropout model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| Dense | (None, 1024) | 1049600 |
| Dense | (None, 1024) | 1049600 |
| Dropout | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [5]:
dropout_model = Sequential()
### START CODE HERE ###
dropout_model.add(Flatten(input_shape=(28,28)))
dropout_model.add(Dense(1024))
dropout_model.add(Dense(1024))
dropout_model.add(Dense(1024))
dropout_model.add(Dropout(0.2))
dropout_model.add(Flatten())
dropout_model.add(Dense(10, activation='softmax'))
### END CODE HERE ###

dropout_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 1024)              803840    
                                                                 
 dense_5 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_6 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 flatten_3 (Flatten)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 10)               

Dropout Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

기존모델과 dropout을 적용한 모델에 대해서, train 데이터와 test데이터에 대한 accuracy차이를 주목해보시면 좋을 것같습니다!

In [6]:
dropout_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

dropout_model_history=dropout_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = dropout_model.evaluate(x_test, y_test, verbose=0)
print('Dropout model Test loss:', score[0])
print('Dropout model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 1s 7ms/step - loss: 2.2954 - accuracy: 0.1515 - val_loss: 2.1920 - val_accuracy: 0.2350
Epoch 2/100
72/72 [==============================] - 0s 6ms/step - loss: 2.1613 - accuracy: 0.2510 - val_loss: 2.0665 - val_accuracy: 0.3800
Epoch 3/100
72/72 [==============================] - 0s 5ms/step - loss: 2.0368 - accuracy: 0.3460 - val_loss: 1.9519 - val_accuracy: 0.4850
Epoch 4/100
72/72 [==============================] - 0s 6ms/step - loss: 1.9192 - accuracy: 0.4510 - val_loss: 1.8480 - val_accuracy: 0.5550
Epoch 5/100
72/72 [==============================] - 0s 6ms/step - loss: 1.8274 - accuracy: 0.5105 - val_loss: 1.7536 - val_accuracy: 0.6050
Epoch 6/100
72/72 [==============================] - 0s 5ms/step - loss: 1.7313 - accuracy: 0.5610 - val_loss: 1.6671 - val_accuracy: 0.6250
Epoch 7/100
72/72 [==============================] - 0s 4ms/step - loss: 1.6415 - accuracy: 0.6025 - val_loss: 1.5879 - val_accuracy: 0.6500
Epoch 8/100
7

##4.3 BatchNormalization
BatchNormalization(BN)은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 BN layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 BN model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [11]:
bn_model = Sequential()
### START CODE HERE ###
bn_model.add(Flatten(input_shape=(28,28)))

bn_model.add(Dense(1024))
bn_model.add(BatchNormalization())
bn_model.add(Activation(activation='relu'))
bn_model.add(Dense(1024))
bn_model.add(BatchNormalization())
bn_model.add(Activation(activation='relu'))
bn_model.add(Dense(1024))
bn_model.add(BatchNormalization())
bn_model.add(Activation(activation='relu'))

bn_model.add(Flatten())
bn_model.add(Dense(10, activation='softmax'))
### END CODE HERE ###

bn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 784)               0         
                                                                 
 dense_17 (Dense)            (None, 1024)              803840    
                                                                 
 batch_normalization_7 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 activation_6 (Activation)   (None, 1024)              0         
                                                                 
 dense_18 (Dense)            (None, 1024)              1049600   
                                                                 
 batch_normalization_8 (Batc  (None, 1024)             4096      
 hNormalization)                                      

BN Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [12]:
bn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
bn_model_history=bn_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = bn_model.evaluate(x_test, y_test, verbose=0)
print('BN model Test loss:', score[0])
print('BN model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 1s 8ms/step - loss: 2.6653 - accuracy: 0.1020 - val_loss: 2.2770 - val_accuracy: 0.1250
Epoch 2/100
72/72 [==============================] - 0s 6ms/step - loss: 2.4562 - accuracy: 0.1420 - val_loss: 2.2465 - val_accuracy: 0.1350
Epoch 3/100
72/72 [==============================] - 0s 6ms/step - loss: 2.2799 - accuracy: 0.1930 - val_loss: 2.1581 - val_accuracy: 0.1800
Epoch 4/100
72/72 [==============================] - 0s 6ms/step - loss: 2.1131 - accuracy: 0.2430 - val_loss: 2.0264 - val_accuracy: 0.2600
Epoch 5/100
72/72 [==============================] - 0s 5ms/step - loss: 1.9639 - accuracy: 0.3115 - val_loss: 1.8914 - val_accuracy: 0.3350
Epoch 6/100
72/72 [==============================] - 0s 5ms/step - loss: 1.8301 - accuracy: 0.3670 - val_loss: 1.7709 - val_accuracy: 0.4150
Epoch 7/100
72/72 [==============================] - 0s 6ms/step - loss: 1.7271 - accuracy: 0.4210 - val_loss: 1.6625 - val_accuracy: 0.4350
Epoch 8/100
7

##4.4 Final Model
지금까지 썼던 Regularization 기법들을 종합선물세트로 적용해 봅시다. 다음과 같은 Final model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dropout | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [10]:
final_model = Sequential()
### START CODE HERE ###
final_model.add(Flatten(input_shape=(28,28)))

final_model.add(Dense(1024))
final_model.add(BatchNormalization())
final_model.add(Activation(activation='relu'))
final_model.add(Dense(1024))
final_model.add(BatchNormalization())
final_model.add(Activation(activation='relu'))
final_model.add(Dense(1024))
final_model.add(BatchNormalization())
final_model.add(Activation(activation='relu'))

final_model.add(Dropout(0.2))
final_model.add(Flatten())
final_model.add(Dense(10, activation='softmax'))
### END CODE HERE ###

final_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_13 (Dense)            (None, 1024)              803840    
                                                                 
 batch_normalization_4 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 1024)              0         
                                                                 
 dense_14 (Dense)            (None, 1024)              1049600   
                                                                 
 batch_normalization_5 (Batc  (None, 1024)             4096      
 hNormalization)                                      

Final Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [13]:
final_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
final_model_history=final_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = final_model.evaluate(x_test, y_test, verbose=0)
print('Final model Test loss:', score[0])
print('Final model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 1s 8ms/step - loss: 2.9090 - accuracy: 0.0935 - val_loss: 2.3280 - val_accuracy: 0.1600
Epoch 2/100
72/72 [==============================] - 0s 5ms/step - loss: 2.7418 - accuracy: 0.1090 - val_loss: 2.3181 - val_accuracy: 0.1600
Epoch 3/100
72/72 [==============================] - 0s 5ms/step - loss: 2.5594 - accuracy: 0.1435 - val_loss: 2.2691 - val_accuracy: 0.1800
Epoch 4/100
72/72 [==============================] - 0s 5ms/step - loss: 2.3936 - accuracy: 0.1935 - val_loss: 2.1780 - val_accuracy: 0.2400
Epoch 5/100
72/72 [==============================] - 0s 5ms/step - loss: 2.2743 - accuracy: 0.2175 - val_loss: 2.0635 - val_accuracy: 0.2700
Epoch 6/100
72/72 [==============================] - 0s 5ms/step - loss: 2.1076 - accuracy: 0.2600 - val_loss: 1.9473 - val_accuracy: 0.3300
Epoch 7/100
72/72 [==============================] - 0s 5ms/step - loss: 2.0067 - accuracy: 0.2865 - val_loss: 1.8365 - val_accuracy: 0.3650
Epoch 8/100
7

(optional) Training data가 늘어나면 regularization 효과가 나는 것을 보였습니다. 쉽게 training data를 늘릴 수 있는 방법은 data augmentation 입니다. 이 방법은 기존 training data를 적절히 rotating, flipping, scaling, shifting 하여 training data 수를 늘리는 것입니다. data augmentation의 regularization 효과를 테스트해 보세요. 또한 뉴럴 네트워크의 노드 개수나 층수를 바꿔서 성능을 올려보는 것도 테스트해보세요.